# 📚 Kapitola 17: Asynchronní programování

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | INTERMEDIATE</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Asyncio event loop**
- **Coroutines a async/await**
- **Tasks a futures**
- **asyncio.gather() a asyncio.wait()**
- **Async context managers**
- **aiohttp pro async HTTP**
- **Async database operations**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 13

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 17

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 17: Asynchronní programování")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Asynchronní programování

## 1. Úvod a motivace

Asynchronní programování je klíčová technologie, kterou každý moderní vývojář potřebuje umět používat – a to zejména v oblastech, kde se zpracování dat zároveň provádí na více místech a často využívá I/O operace (např. sítě, databáze, soubory). V praxi se asynchronní programování využívá například v webových aplikacích, chatbotech, API klientech, v reálném čase zpracovávaných datech (streaming), a hlavně tam, kde výkon a efektivita jsou klíčové.

Když si představíme tradiční synchronní program – např. načtení dat z databáze nebo HTTP požadavek – každý krok čeká, dokud se předchozí neukončí. V případě více žádostí to může výrazně zpomalit celý systém. Asynchronní programování umožňuje spouštět několik operací současně – tedy např. spustit několik HTTP požadavků a čekat na odpovědi, aniž bychom museli počkat na každou zvlášť. Tím se zvýší výkon, sníží se zátěž procesoru a zlepší se reakční doba aplikací.

Student, který se naučí asynchronní programování, bude schopen psát efektivnější a rychlejší aplikace. Bude mít přehled o tom, jak používat `async/await`, `event loop`, `coroutines` a jak správně pracovat s knihovnami jako `aiohttp`, `aiomysql` apod.

---

## 2. Hlavní koncepty

### Asyncio event loop

**Event loop** je jádro asynchronního programování v Pythonu. Je to smyčka, která řídí provádění asynchronních funkcí – tedy `coroutines`. Event loop se stará o plánování a spouštění `coroutines`, přepínání mezi nimi a zajišťuje, aby se všechny operace, které mohou čekat (např. síťové požadavky), neblokovaly celý program.

V Pythonu je event loop standardně automaticky spouštěn při použití `asyncio.run()`:

```python
import asyncio

async def main():
    print("Ahoj")
    
asyncio.run(main())
```

Pokud bychom chtěli spustit více úkolů souběžně, použijeme event loop explicitně nebo prostřednictvím `asyncio.gather()`.

---

### Coroutines a async/await

**Coroutine** je funkce, která může být pozastavena a opět pokračovat – tedy vlastně „asynchronní funkce“.

Používáme klíčová slova `async` a `await`. `async def` označuje asynchronní funkci, která vrací coroutine objekt. `await` se používá uvnitř asynchronní funkce k „pozastavení“ provádění a čekání na dokončení jiného `awaitable` (např. HTTP požadavku, databázového dotazu).

Příklad:

```python
import asyncio

async def fetch_data():
    print("Začínám")
    await asyncio.sleep(1)  # simulace čekání na síť
    print("Dokončeno")
    return "Data"

async def main():
    result = await fetch_data()
    print(result)

asyncio.run(main())
```

V tomto případě `fetch_data()` nezablokuje celý program, ale `await` způsobí, že se program pozastaví a počká na `sleep()`.

---

### Tasks a futures

**Task** je speciální typ `coroutine`, který je plánován pro spuštění v `event loop`. Může být předáván do `asyncio.create_task()` nebo `loop.create_task()`, což umožňuje spouštět více `coroutines` současně.

Futures jsou objekty, které reprezentují výsledek asynchronní operace. Každý `Task` je také `Future`.

```python
async def task1():
    await asyncio.sleep(2)
    return "Výsledek 1"

async def task2():
    await asyncio.sleep(1)
    return "Výsledek 2"

async def main():
    task_a = asyncio.create_task(task1())
    task_b = asyncio.create_task(task2())

    result_a = await task_a
    result_b = await task_b

    print(result_a, result_b)

asyncio.run(main())
```

---

### asyncio.gather() a asyncio.wait()

`gather()` je jednoduchý způsob, jak spustit více `coroutines` současně a počkat na jejich všechny výsledky.

```python
async def fetch(url):
    await asyncio.sleep(1)
    return f"Data z {url}"

async def main():
    results = await asyncio.gather(
        fetch("https://api.example.com/1"),
        fetch("https://api.example.com/2")
    )
    print(results)

asyncio.run(main())
```

`wait()` je pokročilejší nástroj, který umožňuje řídit, jak se mají čekat na `coroutines`. Například můžeme počkat, dokud všechny nedokončí, nebo až některé z nich skončí.

---

### Async context managers

V asynchronním prostředí se také používají **context managers**, které mohou být asynchronní. Místo `with` použijeme `async with`.

```python
import aiohttp

async def fetch_data():
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.example.com") as response:
            return await response.text()
```

Toto zajistí, že zdroje (např. spojení) budou správně uzavřeny i v případě chyby.

---

### aiohttp pro async HTTP

`aiohttp` je knihovna, která umožňuje provádět HTTP požadavky asynchronně. Místo `requests` používáme `aiohttp.ClientSession` a `await`.

```python
import aiohttp
import asyncio

async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    urls = [
        "https://api.github.com/users/octocat",
        "https://api.github.com/users/torvalds"
    ]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        print(results)

asyncio.run(main())
```

---

### Async database operations

V praxi se často provádí i asynchronní databázové operace. Například pomocí `aiomysql`, `asyncpg` nebo `motor` (pro MongoDB). Tyto knihovny umožňují provádět dotazy bez blokování hlavního vlákna.

```python
import aiomysql

async def connect_db():
    conn = await aiomysql.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='password',
        db='mydb'
    )
    
    async with conn.cursor() as cur:
        await cur.execute("SELECT * FROM users")
        result = await cur.fetchall()
        print(result)
    
    conn.close()

asyncio.run(connect_db())
```

---

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout

- **Nepoužití `await`** – Pokud zapomeneš použít `await`, budeš mít objekt `coroutine`, nikoli výsledek.
- **Blokující operace ve smyčce** – Např. `time.sleep()` v asynchronním kódu způsobí, že se celý program zasekne. Použij místo toho `asyncio.sleep()`.
- **Nesprávné použití tasků a futures** – Někdy může být lepší použít `gather()` nebo jinou metodu, než ruční správu tasků.

### Best practices

- Vždy používej `async with` pro asynchronní context managers.
- Používej `asyncio.gather()` nebo `asyncio.wait_for()` pro správné řízení paralelního provádění.
- Dbej na správné zpracování výjimek – asynchronní kód může vyhazovat výjimky, které se musí zachytit.

### Performance tipy

- Používej `asyncio.create_task()` pro spouštění více operací současně.
- Nepoužívej blokující funkce (např. `requests.get()`) v asynchronním kódu.
- Zvaž použití `concurrent.futures.ThreadPoolExecutor` nebo `ProcessPoolExecutor`, pokud potřebuješ provádět CPU náročné operace.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola se zcela přirozeně navazuje na předchozí znalosti o Pythonu, funkcích a jejich práci v rámci `asyncio`. Uživatelé, kteří znají základy asynchronního programování (např. `threading`, `multiprocessing`), si tuto kapitolu mohou snadno přizpůsobit a rozšířit své schopnosti.

Asynchronní programování nejen že rozšiřuje již známé koncepty, ale také je jimi ovlivněno – např. práce s `coroutines` vede k lepšímu porozumění tomu, jak fungují `generators`, a jak se spouštějí asynchronní operace.

---

**Shrnutí:** Asynchronní programování je důležité pro tvorbu efektivních a rychlých aplikací, které využívají síť, databáze a jiné I/O operace bez zbytečného čekání. Práce s `asyncio`, `coroutines`, `tasks`, `aiohttp` a dalšími nástroji umožňuje psát elegantní a efektivní kód – zejména ve webových aplikacích a API klientech.

Připrav se na to, že tato kapitola ti může zásadně změnit způsob, jakým přemýšlíš o výkonu a správě programovacích toků!


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

### **Co příklad demonstruje**:
Základní použití `async` a `await`, vytvoření asynchronní korutiny a jejich spuštění.

### Příklad 2

### **Kompletní spustitelný kód s českými komentáři**:

In [ ]:
import asyncio

# Definice asynchronní korutiny
async def pozdrav(jmeno):
    print(f"Začínám pozdravovat {jmeno}")
    await asyncio.sleep(1)  # Simuluje asynchronní operaci
    print(f"Ahoj {jmeno}!")
    return f"Pozdrav pro {jmeno}"

async def hlavni():
    # Spuštění více korutin
    vysledek1 = await pozdrav("Petr")
    vysledek2 = await pozdrav("Jana")
    
    print(f"Výsledky: {vysledek1}, {vysledek2}")

# Spuštění asynchronního kódu
asyncio.run(hlavni())

### Příklad 3

### **Očekávaný výstup**:
```
Začínám pozdravovat Petr
Ahoj Petr!
Začínám pozdravovat Jana
Ahoj Jana!
Výsledky: Pozdrav pro Petr, Pozdrav pro Jana
```

### Příklad 4

### **Vysvětlení jak kód funguje**:
Klíčové slovo `async` označuje, že funkce je asynchronní korutina. Klíčové slovo `await` způsobí, že se spuštění pozastaví a čeká na dokončení asynchronní operace (`sleep`). `asyncio.run()` spouští celý event loop.

---

## **2. Název příkladu**: Použití `asyncio.gather()`

### Příklad 5

### **Co příklad demonstruje**:
Souběžné spouštění více asynchronních funkcí pomocí `gather()`.

### Příklad 6

### **Kompletní spustitelný kód s českými komentáři**:

In [ ]:
import asyncio

async def stahni_stranku(url):
    print(f"Stahuji {url}")
    await asyncio.sleep(1)
    return f"Obsah z {url}"

async def hlavni():
    # Spustí všechny funkce souběžně
    vysledky = await asyncio.gather(
        stahni_stranku("https://example.com"),
        stahni_stranku("https://python.org"),
        stahni_stranku("https://github.com")
    )
    
    for vysledek in vysledky:
        print(vysledek)

asyncio.run(hlavni())

### Příklad 7

### **Očekávaný výstup**:
```
Stahuji https://example.com
Stahuji https://python.org
Stahuji https://github.com
Obsah z https://example.com
Obsah z https://python.org
Obsah z https://github.com
```

### Příklad 8

### **Vysvětlení jak kód funguje**:
`asyncio.gather()` spouští všechny korutiny souběžně a čeká na jejich dokončení. Výsledky se vrátí jako seznam ve stejném pořadí, jak byly zadány.

---

## **3. Název příkladu**: Asynchronní HTTP požadavky s `aiohttp`

### Příklad 9

### **Co příklad demonstruje**:
Asynchronní HTTP požadavky pomocí knihovny `aiohttp`.

### Příklad 10

### **Kompletní spustitelný kód s českými komentáři**:

In [ ]:
import asyncio
import aiohttp

async def stahni_stranku(session, url):
    async with session.get(url) as odpoved:
        obsah = await odpoved.text()
        return f"{url}: {len(obsah)} znaků"

async def hlavni():
    urls = [
        'https://httpbin.org/delay/1',
        'https://httpbin.org/delay/2',
        'https://httpbin.org/delay/1'
    ]

    async with aiohttp.ClientSession() as session:
        vysledky = await asyncio.gather(*[stahni_stranku(session, url) for url in urls])
        for vysledek in vysledky:
            print(vysledek)

asyncio.run(hlavni())

### Příklad 11

### **Očekávaný výstup**:
```
https://httpbin.org/delay/1: 389 znaků
https://httpbin.org/delay/2: 389 znaků
https://httpbin.org/delay/1: 389 znaků
```

### Příklad 12

### **Vysvětlení jak kód funguje**:
`aiohttp.ClientSession()` vytvoří klienta pro asynchronní HTTP požadavky. Používá `async with` pro správné uzavření připojení. `asyncio.gather()` spouští požadavky souběžně, což znamená, že se nečeká na každý požadavek jednotlivě.

---

## **4. Název příkladu**: Asynchronní databázové operace

### Příklad 13

### **Co příklad demonstruje**:
Simulace asynchronní práce s databází (např. `aiomysql` nebo `asyncpg`).

### Příklad 14

### **Kompletní spustitelný kód s českými komentáři**:

In [ ]:
import asyncio

# Simulace asynchronní databázové operace
async def nacti_uzivatele(user_id):
    print(f"Načítám uživatele ID: {user_id}")
    await asyncio.sleep(0.5)  # Simuluje čekání na databázi
    return {"id": user_id, "jmeno": f"Uživatel_{user_id}"}

async def hlavni():
    uzivatele = [1, 2, 3]
    
    # Spustí všechny dotazy souběžně
    vysledky = await asyncio.gather(*[nacti_uzivatele(uid) for uid in uzivatele])
    
    for uzivatel in vysledky:
        print(f"Nalezen uživatel: {uzivatel}")

asyncio.run(hlavni())

### Příklad 15

### **Očekávaný výstup**:
```
Načítám uživatele ID: 1
Načítám uživatele ID: 2
Načítám uživatele ID: 3
Nalezen uživatel: {'id': 1, 'jmeno': 'Uživatel_1'}
Nalezen uživatel: {'id': 2, 'jmeno': 'Uživatel_2'}
Nalezen uživatel: {'id': 3, 'jmeno': 'Uživatel_3'}
```

### Příklad 16

### **Vysvětlení jak kód funguje**:
Tento příklad simuluje asynchronní práci s databází – každý dotaz na uživatele trvá nějakou dobu a všechny se spouští paralelně. V praxi by zde byla reálná databázová operace.

---

## **5. Název příkladu**: Asynchronní kontextový manažér

### Příklad 17

### **Co příklad demonstruje**:
Použití `async with` pro správu asynchronních zdrojů (např. souborů, databází).

### Příklad 18

### **Kompletní spustitelný kód s českými komentáři**:

In [ ]:
import asyncio

class AsyncResource:
    def __init__(self, nazev):
        self.nazev = nazev

    async def __aenter__(self):
        print(f"Otevírám zdroj: {self.nazev}")
        await asyncio.sleep(0.1)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print(f"Uzavírám zdroj: {self.nazev}")
        await asyncio.sleep(0.1)

async def hlavni():
    async with AsyncResource("Soubor_1") as res:
        print(f"Používám zdroj: {res.nazev}")

    async with AsyncResource("Soubor_2") as res:
        print(f"Používám zdroj: {res.nazev}")

asyncio.run(hlavni())

### Příklad 19

### **Očekávaný výstup**:
```
Otevírám zdroj: Soubor_1
Používám zdroj: Soubor_1
Uzavírám zdroj: Soubor_1
Otevírám zdroj: Soubor_2
Používám zdroj: Soubor_2
Uzavírám zdroj: Soubor_2
```

### Příklad 20

### **Vysvětlení jak kód funguje**:
Asynchronní kontextový manažér používá `__aenter__` a `__aexit__`, které jsou volány při vstupu a ukončení bloku `async with`. Tento způsob zajišťuje správné otevírání a uzavírání asynchronních zdrojů.

---

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# 5 praktických cvičení pro kapitolu: Asynchronní programování

---

## Cvičení 1: **Asynchronní HTTP požadavky s `async/await`**

### 1. **Název úkolu**
Asynchronní stahování dat z API

### 2. **Detailní zadání**
Vytvořte program, který paralelně načte data ze tří různých API endpointů (např. JSONPlaceholder, OpenWeatherMap, nebo jiné veřejné API) a vytiskne výsledky. Použijte `async/await` pro řízení asynchronních operací.

### 3. **Vstupní data/požadavky**
- Využijte libovolné veřejné API (např. [JSONPlaceholder](https://jsonplaceholder.typicode.com/), [OpenWeatherMap](https://openweathermap.org/api))
- Načtěte data z minimálně tří endpointů.
- Všechny požadavky běží paralelně, ne postupně.

### 4. **Očekávaný výstup**
- Výstup obsahuje data z každého API endpointu (např. uživatelské jméno, počasí, atd.)
- Zpracované výsledky se vytisknou do konzole bez závislosti na pořadí provádění.

### 5. **Hints/Nápověda**
1. Použijte `fetch()` pro HTTP požadavky.
2. Využijte `Promise.all()` nebo `Promise.allSettled()` pro paralelní spuštění požadavků.
3. Zpracujte výsledky při použití `async/await`.
4. Nezapomeňte na zpracování chyb (try/catch).

### 6. **Kostra řešení**
```javascript
async function fetchData() {
    try {
        // Zde implementujte paralelní volání API
    } catch (error) {
        console.error('Chyba při načítání dat:', error);
    }
}

fetchData();
```

### 7. **Bonusové rozšíření**
- Přidejte časový limit pro každý požadavek pomocí `Promise.race()`.
- Zapište výsledky do souboru (např. JSON) místo konzole.

---

## Cvičení 2: **Asynchronní práce s soubory**

### 1. **Název úkolu**
Čtení a zápis souborů v asynchronním režimu

### 2. **Detailní zadání**
Vytvořte program, který asynchronně čte více textových souborů, spojuje jejich obsah a zapisuje do nového souboru. Použijte `fs.promises` pro práci se soubory.

### 3. **Vstupní data/požadavky**
- Vstupní soubory: `file1.txt`, `file2.txt`, `file3.txt`
- Každý soubor obsahuje jednu řádku textu.
- Výsledný soubor bude `output.txt` a obsahuje spojené řádky.

### 4. **Očekávaný výstup**
- Soubor `output.txt` obsahující spojený text z všech vstupních souborů.
- V konzoli se zobrazí potvrzení, že byl soubor úspěšně vytvořen.

### 5. **Hints/Nápověda**
1. Použijte `fs.readFile()` pro čtení souborů.
2. Využijte `Promise.all()` pro paralelní čtení souborů.
3. Po načtení všech obsahů použijte `fs.writeFile()` k zápisu do nového souboru.
4. Přidejte logiku pro ošetření chyb při čtení nebo zápisu.

### 6. **Kostra řešení**
```javascript
const fs = require('fs').promises;

async function processFiles() {
    try {
        // Čtení souborů
        // Zápis do výstupního souboru
    } catch (error) {
        console.error('Chyba při zpracování souborů:', error);
    }
}

processFiles();
```

### 7. **Bonusové rozšíření**
- Vytvořte program, který zpracuje složku a všechny `.txt` soubory uvnitř.
- Přidejte možnost volby oddělovače mezi řádky (např. nový řádek, středník, atd.).

---

## Cvičení 3: **Asynchronní zpracování dat s `setTimeout` a `Promise`**

### 1. **Název úkolu**
Simulace asynchronního zpracování dat s pomocí `setTimeout`

### 2. **Detailní zadání**
Vytvořte funkci, která přijímá pole čísel a pro každé číslo asynchronně vypočítá jeho druhou mocninu (po zpoždění 100ms). Využijte `setTimeout` v kombinaci s `Promise`.

### 3. **Vstupní data/požadavky**
- Vstupní pole: `[2, 4, 5, 7]`
- Každá operace trvá 100ms.
- Výsledek musí být zpracován asynchronně.

### 4. **Očekávaný výstup**
- Pole výsledků: `[4, 16, 25, 49]`
- Výstup se zobrazí po splnění všech operací.

### 5. **Hints/Nápověda**
1. Vytvořte pomocnou funkci, která používá `setTimeout` a `Promise`.
2. Použijte `Promise.all()` pro paralelní zpracování všech čísel.
3. Využijte `map()` k iteraci přes pole.
4. Ošetřete chyby pomocí `catch()`.

### 6. **Kostra řešení**
```javascript
function squareWithDelay(num) {
    return new Promise(resolve => {
        setTimeout(() => resolve(num * num), 100);
    });
}

async function processNumbers(numbers) {
    try {
        // Zde implementujte zpracování čísel
    } catch (error) {
        console.error('Chyba:', error);
    }
}
```

### 7. **Bonusové rozšíření**
- Přidejte možnost nastavit různé zpoždění pro každé číslo.
- Simulujte chybu při zpracování jednoho z čísel a ošetřete ji.

---

## Cvičení 4: **Asynchronní generátor s `async/await`**

### 1. **Název úkolu**
Vytvoření async generátoru pro postupné načítání dat

### 2. **Detailní zadání**
Implementujte async generátor, který postupně vrací data (např. z API nebo pole) po určitém časovém intervalu. Použijte `async/await` k řízení toku.

### 3. **Vstupní data/požadavky**
- Vstupní pole: `[10, 20, 30, 40]`
- Každý prvek se vrací po 500ms.
- Použijte `for await...of` pro iteraci.

### 4. **Očekávaný výstup**
- V konzoli se postupně (po 500ms) tisknou hodnoty: `10


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: Asynchronní programování

## 📝 Články a tutoriály

1. **[Understanding Async/Await in JavaScript](https://developer.mozilla.org/en-US/docs/Learn/JavaScript/Asynchronous/Async_await)**  
   Oficiální dokumentace MDN pro async/await v JavaScriptu. Výklad základních konceptů, praxe a porovnání s promises.

2. **[A Complete Guide to Asynchronous Programming](https://css-tricks.com/async-and-await/)**  
   Rozsáhlý článek o asynchronním programování v JavaScriptu, zaměřený na práci s async/await a řešení běžných problémů.

3. **[Concurrency in Go](https://golang.org/doc/effective_go#concurrency)**  
   Efektivní používání concurrency v jazyce Go – základy gorutin, kanálů a synchronizace.

4. **[Python Asyncio Documentation](https://docs.python.org/3/library/asyncio.html)**  
   Oficiální dokumentace k asyncio v Pythonu – komplexní vysvětlení základních konceptů jako event loop, tasks a coroutines.

5. **[Understanding JavaScript Promises](https://javascript.info/promise-basics)**  
   Výukový článek o promise v JavaScriptu, včetně řešení chyb a praxe s `.then()`, `.catch()` a `async/await`.

---

## 🎥 YouTube videa

1. **[JavaScript Async/Await Explained - Full Tutorial](https://www.youtube.com/watch?v=5lQZ56W09z4)** – 15 minut  
   Detailní video s vysvětlením async/await ve středně pokročilém jazyce JavaScript. Ukazuje příklady z praxe.

2. **[Go Concurrency Patterns](https://www.youtube.com/watch?v=f6kdp273VUg)** – 48 minut  
   Výukové video od Rob Pike o gorutinách, kanálech a konceptech concurrency v Go.

3. **[Python asyncio - Event Loop and Coroutines Explained](https://www.youtube.com/watch?v=9Bb5131lZ20)** – 22 minuty  
   Výklad principů asyncio v Pythonu – event loop, tasky a coroutines s praktickými příklady.

---

## 📚 Knihy a oficiální dokumentace

1. **[Effective Go - Concurrency](https://golang.org/doc/effective_go#concurrency)**  
   Oficiální dokumentace jazyka Go s rozsáhlým vysvětlením concurrency – gorutiny, kanály a synchronizace.

2. **[Python Tricks: The Book](https://www.realpython.com/products/python-tricks-book/)**  
   Kniha s kapitolou o asynchronním programování v Pythonu, zahrnující asyncio, coroutines a event loop.

3. **[JavaScript: The Good Parts](https://www.amazon.com/JavaScript-Good-Parts-Douglas-Crockford/dp/0596517742)**  
   Klasická kniha o JavaScriptu – výklad asynchronních konceptů, promise a event loop.

---

## 💻 Praktické projekty

1. **Napiš jednoduchou webovou aplikaci s async/await (JavaScript)**  
   Vytvoř jednoduchý REST API client, který načítá data z externího API a předává je uživateli pomocí `async/await`.

2. **Vytvoř asynchronní server v Pythonu s asyncio**  
   Napiš jednoduchý HTTP server, který zvládne více požadavků současně pomocí asyncio.

---

## 🧰 Online nástroje a playgroundy

1. **[JavaScript Async Playground](https://replit.com/@javascript/async-await-playground)**  
   Interaktivní online prostředí pro experimentování s async/await v JavaScriptu.

2. **[Python Asyncio Playground](https://repl.it/@python/asyncio-playground


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Asyncio event loop</li>
<li>✓ Coroutines a async/await</li>
<li>✓ Tasks a futures</li>
<li>✓ asyncio.gather() a asyncio.wait()</li>
<li>✓ Async context managers</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 18 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:28:53*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

